In [1]:
import sys
sys.path.append('../script/')
import os
from os.path import exists
from datetime import datetime
import json
import gc
from functools import partial

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from adabelief_pytorch import AdaBelief
import optuna

import utils
import models
import train as trainer
DEVICE = "cuda"
EPOCHS = 3000
MODELNAME = "Baseline1103"
if not exists(MODELNAME):
    os.makedirs(f"{MODELNAME}/tensorboard")
now = datetime.now()
now = str(now)[5:17].replace(" ", "_").replace(":", "")
writer = SummaryWriter(log_dir=f"{MODELNAME}/tensorboard")

ModuleNotFoundError: No module named 'torch'

In [ ]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")
targets = targets[:-1]

In [ ]:
class BaseLine(nn.Module):
    def __init__(self, num_features, num_targets, num_layers=3, dropout=.2, hidden_size=256, activation="relu", batchnorm=True):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.Linear(num_features if len(layers)==0 else hidden_size, hidden_size, bias=(not batchnorm)))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            if activation == "relu":
                layers.append(nn.ReLU())
            elif activation == "prelu":
                layers.append(nn.PReLU())
            else:
                raise RuntimeError(f'{activation} is not implemented')
        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def set_output_bias(model, df, targets):   
    init_bias = np.array([])
    for target in targets:
        try:
            neg, pos = np.bincount(df[target])
        except ValueError:
            neg, pos = np.array([df.shape[0], 0.01])
        init_bias_ = np.log([pos/neg])
        init_bias = np.append(init_bias, init_bias_)
    model.model[-1].bias.data = torch.tensor(init_bias, dtype=torch.float32)
    return model

In [ ]:
def run_training(df, fold, params, hp_tune=False):

    save_model = False if hp_tune else True
    print(f'\n[Fold No.{fold:>3}]')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)

    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    model = set_output_bias(model, df, targets)
    model.to(DEVICE)

    if params["optimizer"] == "SGD":
        # params["optim_params"]["momentum"] = trial.suggest_uniform("momentum", 0.1, 0.8)
        # params["optim_params"]["momentum"] = trial.suggest_categorical("nesterov", [True, False])
        optimizer = torch.optim.SGD(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "Adam":
        # params["optim_params"]["momentum"] = trial.suggest_uniform("momentum", 0.1, 0.8)
        # params["optim_params"]["amsgrad"] = trial.suggest_categorical("amsgrad", [True, False])
        optimizer = torch.optim.Adam(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "AdamW":
        # params["optim_params"]["momentum"] = trial.suggest_uniform("momentum", 0.1, 0.8)
        # params["optim_params"]["amsgrad"] = trial.suggest_categorical("amsgrad", [True, False])
        optimizer = torch.optim.AdamW(model.parameters(), **params["optim_params"])
    elif params["optimizer"] == "AdaBelief":
        optimizer = AdaBelief(model.parameters(), **params["optim_params"])
    else:
        raise RuntimeError(f'{params["optimizer"]} is not implemented')

    if params["scheduler"] == "ReduceLROnPlateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", **params["scdl_params"])
    else: 
        print("Not Implemented: No scheduling will be applied")
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda ep: 1**ep)

    eng = utils.Engine(model, optimizer, device=DEVICE)

    del df, train_df, valid_df, x_tr, x_va, y_tr, y_va
    gc.collect()

    torch.backends.cudnn.benchmark = True

    print(f'Training state is shown in {MODELNAME}/tensorboard')
    filename = f"{MODELNAME}/{now}_fold{fold}.pt"

    loss_best = np.inf
    patience = 25
    patience_cnt = 0
    for ep in range(EPOCHS):
        print(f'Ep.{ep:>3}/{EPOCHS:>3}, patience:{patience_cnt:>2}/{patience:>2}', end='\r')
        loss_tr = eng.train(loader_tr)
        loss_va = eng.validate(loader_va)
        scheduler.step(loss_va)
        writer.add_scalars(f'{now}/fold{fold}', {'train':loss_tr, 'valid':loss_va}, ep)
        if loss_va < loss_best:
            patience_cnt = 0
            loss_best = loss_va
            if save_model:
                torch.save(model.model.state_dict(), filename)
        else:
            patience_cnt += 1
        if patience_cnt > patience:
            break

    print("\nmodel saved at:", filename)

In [ ]:
params = {
    "nn_params": {"dropout": 0.5, "num_layers": 4, "hidden_size": 512, "activation": "relu", "batchnorm": True},
    "optimizer": "AdaBelief",
    "optim_params": {"lr":1e-2, "eps":1e-16, "betas":(0.9,0.999), "weight_decouple":True, "rectify":True},
    "scheduler": "ReduceLROnPlateau",
    "scdl_params": {"threshold": 0.00001, "patience": 3}
}
# 0.02355, 0.03 on momentum:0

In [ ]:
%%time
for fold in range(4):
    run_training(df, fold, params)

In [ ]:
with open(f'{MODELNAME}/{now}_params.json', 'w') as f:
    json.dump(params, f, indent=4)

In [ ]:
writer.close()

In [ ]:
gc.collect()

---
Hyper parameter optimization

In [ ]:
def objective(trial):
    params = {
        "nn_params": {"dropout": trial.suggest_uniform("dropout", 0.1, 0.8), 
                      "num_layers": trial.suggest_int("num_layers", 1, 7),
                      "hidden_size": trial.suggest_int("hidden_size", 16, 2048),
                      "activation": trial.suggest_categorical("activation", ["relu", "prelu"]),
                      "batchnorm": trial.suggest_categorical("batchnorm", [True, False])},
        "optimizer": trial.suggest_categorical("optimizer", ["SGD", "Adam"]),
        "optim_params": {"lr": trial.suggest_loguniform("lr", 1e-6, 1e-3)},
        "scheduler": "ReduceLROnPlateau",
        "scdl_params": {"threshold": 0.00001},
    }
    loss_all = []
    for fold_ in range(4):
        loss_tmp = run_training(df, fold, params, save_model=False)
        loss_all.append(loss_tmp)
    return np.mean(loss_all)